In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer,BertForSequenceClassification
from sklearn.metrics import classification_report

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Loading data

In [3]:
data=pd.read_csv(r'C:\Users\DELL\Documents\MAJOR PROJECT\evaluation.csv')

In [4]:
data.head()

,similarity,sentence1,sentence2
0,1,two women embracing holding go packages,sisters hugging goodbye holding go packages ea...
1,2,two women embracing holding go packages,two woman holding packages
2,0,two women embracing holding go packages,men fighting outside deli
3,2,two young children blue jerseys one number 9 o...,two kids numbered jerseys wash hands
4,1,two young children blue jerseys one number 9 o...,two kids ballgame wash hands


In [ ]:
sent_pairs=[(row['sentence1'],row['sentence2']) for index,row in data.iterrows()]

Loading tokenizer and selected model from phase 1 

In [14]:
model=BertForSequenceClassification.from_pretrained(r'C:\Users\DELL\Documents\MAJOR PROJECT\model2',num_labels=3)
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

Applying Sliding Window Attention on data

In [19]:
def sliding_window_mask(input_ids,window_size):
  batch_size,seq_len=input_ids.shape
  mask=torch.ones((batch_size,seq_len),dtype=torch.float)
  for batch in range(batch_size):
    for i in range(seq_len):
      start=max(0,i-window_size//2)
      end=min(seq_len,i+(window_size+1)//2)
      row_mask=torch.zeros(seq_len,dtype=torch.float)
      row_mask[start:end]=1
      mask[batch,i]=row_mask[i]
  return mask

marks=[]
sims=[]
for sent1,sent2 in sent_pairs:
  inputs=tokenizer(sent1,sent2,return_tensors='pt',padding=True,truncation=True)
  input_ids=inputs['input_ids']
  window_size=7
  attention_mask=sliding_window_mask(input_ids,window_size)
  outputs=model(input_ids,attention_mask=attention_mask,labels=torch.tensor([1]))
  logits=outputs.logits
  mark_obtained=torch.sigmoid(logits)
  sim=torch.argmax(logits,dim=1).item()
  if sim==0:
    avg_marks=torch.min(mark_obtained).item()
  elif sim==1:
    avg_marks=torch.mean(mark_obtained).item()
  elif sim==2:
    avg_marks=torch.max(mark_obtained).item()
  marks.append(avg_marks)
  sims.append(sim)

Evaluation

In [ ]:
report=classification_report(list(data['similarity']),sims)
rep_lines=report.split('\n')
rep_lines=rep_lines[2:]
df=[]
for line in rep_lines:
  if line.strip():
    row=line.strip().split()
    df.append({'class':row[0],'precision':row[1],'recall':row[2]})
print('Classification Report for Model with SW'+'\n',pd.DataFrame(df))

Classification Report for Model with SW
       class precision recall
0         0      0.93   0.87
1         1      0.81   0.87
2         2      0.90   0.89
3  accuracy      0.88   1000
4     macro       avg   0.88
5  weighted       avg   0.88
